In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, merge, UpSampling2D
from keras.layers.merge import concatenate

import cv2
from tqdm import tqdm


Using Theano backend.


In [83]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = all_info[0]
x_test = all_info[1]
y_train = all_info[2]
y_test = all_info[3]

In [2]:
def get_unet():
    inputs1 = Input(shape=(32, 32, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs1)
    conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv12)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv23 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv23)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv32 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv32)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv42 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv42)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv52 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv52), conv42], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv62 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv62), conv32], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv72 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv72), conv23], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv82 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv82), conv12], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv92 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    flat = Flatten()(conv92)
    dense1 = Dense(128, activation='relu')(flat)
    drop = Dropout(0.5)(dense1)
    dense2 = Dense(17, activation='sigmoid')(drop)

    model = Model(inputs=inputs1, outputs=dense2)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [3]:
model = get_unet()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 32)    896         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 32, 32)    9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 16, 16, 32)    0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [89]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(35000, 32, 32, 3)
(5479, 32, 32, 3)
(35000, 17)
(5479, 17)


In [93]:
print (k.__version__)
model = get_unet()

model.fit(x_train, y_train, batch_size=128, epochs=8, verbose=1,
         validation_data=(x_test, y_test))

from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

2.0.4
Train on 35000 samples, validate on 5479 samples
Epoch 1/8
35000/35000 [==============================] - 2223s - loss: 0.2750 - acc: 0.8957 - val_loss: 0.2348 - val_acc: 0.9078
Epoch 2/8
35000/35000 [==============================] - 2494s - loss: 0.2343 - acc: 0.9101 - val_loss: 0.2215 - val_acc: 0.9106
Epoch 3/8
35000/35000 [==============================] - 2516s - loss: 0.2229 - acc: 0.9143 - val_loss: 0.2098 - val_acc: 0.9158
Epoch 4/8
35000/35000 [==============================] - 2249s - loss: 0.2146 - acc: 0.9163 - val_loss: 0.2008 - val_acc: 0.9187
Epoch 5/8
35000/35000 [==============================] - 2252s - loss: 0.2045 - acc: 0.9185 - val_loss: 0.1882 - val_acc: 0.9241
Epoch 6/8
35000/35000 [==============================] - 2277s - loss: 0.1972 - acc: 0.9224 - val_loss: 0.1830 - val_acc: 0.9273
Epoch 7/8
35000/35000 [==============================] - 2279s - loss: 0.1899 - acc: 0.9264 - val_loss: 0.1778 - val_acc: 0.9298
Epoch 8/8
35000/35000 [===================

In [94]:
from keras.models import load_model

Turns out you need to name all the layers differently if you want to save the model...
See Unet Saving Test if you want an example
Anyway, for now I'll just save the weights only ._.

In [98]:
model.save_weights('unets/unet_1_weights.h5')